In [1]:
print("hello")

hello


In [8]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib
import os

In [14]:
df = pd.read_csv('/Users/swatea/Elderly-Care/data/daily_reminder.csv')
df.head()

,Device-ID/User-ID,Timestamp,Reminder Type,Scheduled Time,Reminder Sent (Yes/No),Acknowledged (Yes/No),Unnamed: 6
0,D1000,1/2/2025 11:25,Exercise,13:00:00,No,No,NaN
1,D1001,1/3/2025 2:52,Hydration,13:00:00,Yes,Yes,NaN
2,D1002,1/8/2025 13:50,Appointment,13:30:00,No,No,NaN
3,D1003,1/5/2025 5:16,Exercise,8:00:00,No,No,NaN
4,D1004,1/1/2025 4:20,Medication,11:30:00,No,No,NaN


In [20]:
# Step 3: Preprocess time + categorical columns
df["Scheduled Time"] = pd.to_datetime(df["Scheduled Time"], format='%H:%M:%S', errors='coerce')
df["minutes_since_midnight"] = df["Scheduled Time"].dt.hour * 60 + df["Scheduled Time"].dt.minute

# Encode yes/no flags
df["reminder_sent_flag"] = df["Reminder Sent (Yes/No)"].map({"Yes": 1, "No": 0})
df["acknowledged_flag"] = df["Acknowledged (Yes/No)"].map({"Yes": 1, "No": 0})

# Encode Reminder Type
le = LabelEncoder()
df["reminder_type_encoded"] = le.fit_transform(df["Reminder Type"])

df.head()

,Device-ID/User-ID,Timestamp,Reminder Type,Scheduled Time,Reminder Sent (Yes/No),Acknowledged (Yes/No),hour_of_day,reminder_sent_flag,acknowledged_flag,reminder_type_encoded,minutes_since_midnight
0,D1000,1/2/2025 11:25,Exercise,1900-01-01 13:00:00,No,No,13,0,0,1,780
1,D1001,1/3/2025 2:52,Hydration,1900-01-01 13:00:00,Yes,Yes,13,1,1,2,780
2,D1002,1/8/2025 13:50,Appointment,1900-01-01 13:30:00,No,No,13,0,0,0,810
3,D1003,1/5/2025 5:16,Exercise,1900-01-01 08:00:00,No,No,8,0,0,1,480
4,D1004,1/1/2025 4:20,Medication,1900-01-01 11:30:00,No,No,11,0,0,3,690


In [21]:
X = df[["reminder_type_encoded", "minutes_since_midnight", "reminder_sent_flag"]]
y = df["acknowledged_flag"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.84      0.91      1820
           1       0.80      1.00      0.89      1180

    accuracy                           0.90      3000
   macro avg       0.90      0.92      0.90      3000
weighted avg       0.92      0.90      0.90      3000

